In [1]:
import numpy as np
import sklearn as sk  # namespace conflict with pyspark
import sklearn.ensemble as skEnsemble  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from helper import *
import tensorflow as tf

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator

# from keras.layers import Dense
# from keras.models import Model, Sequential
# from keras import initializers, optimizers
# from keras import backend as K

# Stuff that needs to be done

- Find another framework.
    - Sklearn
    - Spark
    - Tensorflow/R
- Algorithms
    - Linear Regression
    - Logistic Regression
    - Maybe SVM /Random forest
- Find two metrics 
    - RMSE 
    - Some other
- Find another dataset 
    - Sum data with noise 
    -  Maybe housing data set
    

# Scikitlearn


## Import DataSet

In [2]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [3]:
get_data(all_urls) # retrieves the data if there is no data folder

In [4]:
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/kc_house_data.csv" # has more than 30 features
# need one more
# what a brilliant idea to name files with space

## Load datasets sum

In [5]:
sumdata = pd.read_csv(sumdata_path, delimiter=";", nrows=100)
sumdata.head(n=10)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Target,Target Class
0,1,57326,68791,82549,99059,72624,142645,171174,205409,246491,295789,1369233,Very Large Number
1,2,87859,105431,126517,151820,19982,218621,262345,314814,377777,453332,2098516,Very Large Number
2,3,23721,28465,34158,40990,20054,59026,70831,84997,101996,122395,566579,Very Large Number
3,4,24771,29725,35670,42804,7775,61638,73966,88759,106511,127813,591657,Very Large Number
4,5,47862,57434,68921,82705,60872,119095,142914,171497,205796,246955,1143179,Very Large Number
5,6,35286,42343,50812,60974,51392,87803,105364,126437,151724,182069,842812,Very Large Number
6,7,14070,16884,20261,24313,1509,35011,42013,50416,60499,72599,336066,Large Number
7,8,34018,40822,48986,58783,38750,84648,101578,121894,146273,175528,812530,Very Large Number
8,9,36379,43655,52386,62863,29843,90523,108628,130354,156425,187710,868923,Very Large Number
9,10,25867,31040,37248,44698,92630,64366,77239,92687,111224,133469,617838,Very Large Number


## Preprocess sum dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract rest of the columns as explananatory variables, except Instance and Target, Target Class
- Ensure all dataframe has been converted to numpy array


In [6]:
# Remove 'Instance' as it simply represents the row number

sumdata.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_reg_Y = sumdata.loc[:, ['Target']]

# Extract rest columns as explananatory variables
sumdata_reg_X =sumdata.drop(['Instance','Target Class','Target'
                                    ], axis = 1)


## Load datasets house price

In [7]:
housing_price = pd.read_csv(housing_price_path, delimiter=",")
housing_price.head(n=10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


## Preprocess house price dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest of the columns as explananatory variables

- Ensure all dataframe has been converted to numpy array


In [8]:
# Remove 'Instance' as it simply represents the row number

# Extract 'Nosiy Target' as regression target
housing_price_reg_Y = housing_price.loc[:, ['price']]

# get all the numerical data that is needed
explanatory_numeric = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'condition', 'sqft_above', 'yr_built',
                      'sqft_living15', 'sqft_lot15']

housing_price[explanatory_numeric[0]]
filtered_table = housing_price[explanatory_numeric[0]]
filtered_table

for c in explanatory_numeric[1:]:
    filtered_table = pd.concat([filtered_table, housing_price[c]], axis=1)
filtered_table

# get feature
housing_price_reg_X = filtered_table[:]

# get target
housing_price_reg_Y = housing_price['price']






In [9]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


In [10]:
def spark_random_forest( X_train, X_test, y_train, y_test):
    """
        numTrees = 10\n",
        maxDepth = 30
        minInfoGain =2
    """
    from pyspark.ml import Pipeline
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    train_data = pd.concat([X_train,y_train], axis =1)
    test_data  = pd.concat([X_test,y_test], axis =1)
    
 
    train_data = sqlContext.createDataFrame(train_data)
    test_data = sqlContext.createDataFrame(test_data) 
   
   
    assembler = (VectorAssembler()
        .setInputCols(train_data.columns[:-1]) # everything excluding target
        .setOutputCol("features"))
 
    rfr = pyspark.ml.regression.RandomForestRegressor(labelCol=train_data.columns[-1], numTrees=10, maxDepth=5, minInfoGain=1)

    pipeline = Pipeline(stages=[assembler, rfr])
    model = pipeline.fit(train_data)
    
    
    predictions = model.transform(test_data) 
    
    #predictions.select("predictions", "label").toPandas()  
    evaluator = RegressionEvaluator(metricName="rmse", labelCol=train_data.columns[-1])
    RMSE = evaluator.evaluate(predictions)
    evaluator = RegressionEvaluator(metricName="mae", labelCol=train_data.columns[-1])
    MAE = evaluator.evaluate(predictions)
  
    return RMSE,MAE
    

In [11]:
def sklearn_random_forest( X_train, X_test, y_train, y_test): 
    """
        n_estimators 10\n",
        max_depth 30, 
        criterion mae\n",
        min_samples_split 0
    """

    rfr = skEnsemble.RandomForestRegressor(n_estimators=10, max_depth=5, min_samples_split=1.0)
    model = rfr.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae =  sk.metrics.mean_absolute_error(y_test, preds)
    rmse = sk.metrics.mean_squared_error(y_test, preds)**0.5
    return (rmse,mae)

### 70/30 split 

In [12]:
def model_test_split( data_x, data_y, dataset_name, algorithm): 
    algorithm_n = algorithm.__name__

    X = data_x
    y = data_y

    kf = KFold(n_splits=10)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)


        # accumulating errors for each fold
    error = algorithm( X_train, X_test, y_train, y_test) 

    print ("Algorithm: {}  Data set: {} Error: {}".format(algorithm_n, dataset_name, error))
        

In [13]:
model_test_split( sumdata_reg_X, sumdata_reg_Y, "Sum data with noise", spark_random_forest)


Algorithm: spark_random_forest  Data set: Sum data with noise Error: (63104.681180980806, 52072.28320634917)


In [14]:
model_test_split( sumdata_reg_X, sumdata_reg_Y, "Sum data with noise", sklearn_random_forest)

Algorithm: sklearn_random_forest  Data set: Sum data with noise Error: (613110.27289336745, 544754.19419047621)


/Users/kiyim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


In [15]:
model_test_split( housing_price_reg_X, housing_price_reg_Y, "House Price", spark_random_forest)

Algorithm: spark_random_forest  Data set: House Price Error: (239398.1136284069, 152047.40558890987)


In [16]:
model_test_split( housing_price_reg_X, housing_price_reg_Y, "House Price", sklearn_random_forest)

Algorithm: sklearn_random_forest  Data set: House Price Error: (373107.31535735331, 236030.42221635659)
